In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import pymysql
import datetime
from prophet import Prophet

In [3]:
data = pd.read_csv('./소고기가격_2016_2022_11월17일까지.csv',index_col=0)
data

,date,도매가격
1706,2016-01-04,17343.0
1705,2016-01-05,18445.0
1704,2016-01-06,18813.0
1703,2016-01-07,18905.0
1702,2016-01-08,18950.0
...,...,...
15,2022-11-16,17832.0
15,2022-11-16,17832.0
16,2022-11-17,17331.0
16,2022-11-17,17331.0


In [4]:
mydb = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '1234',
    db = 'price',
    charset='utf8'
)

cursor = mydb.cursor(pymysql.cursors.DictCursor)
cursor.execute("SELECT * FROM price.cow ORDER BY 1 DESC LIMIT 5")
result = cursor.fetchall()

for row in result :
    print(row)

{'date': '2022-11-25', '도매가격': 16309}
{'date': '2022-11-24', '도매가격': 16570}
{'date': '2022-11-23', '도매가격': 16951}
{'date': '2022-11-22', '도매가격': 17102}
{'date': '2022-11-21', '도매가격': 13000}


In [5]:
# 조회 결과 데이터프레임으로 저장
df = pd.DataFrame(result)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date',ascending=True)
df

,date,도매가격
4,2022-11-21,13000
3,2022-11-22,17102
2,2022-11-23,16951
1,2022-11-24,16570
0,2022-11-25,16309


In [6]:
days_1 = pd.DataFrame(df['date'][-1:]  + datetime.timedelta(days=1))
days_2 = pd.DataFrame(df['date'][-1:]  + datetime.timedelta(days=2))
days = pd.concat((days_1,days_2))
days

,date
0,2022-11-26
0,2022-11-27


In [7]:
df = pd.merge(df,days,on='date',how='outer')
df = df.fillna(method='ffill')
df

,date,도매가격
0,2022-11-21,13000.0
1,2022-11-22,17102.0
2,2022-11-23,16951.0
3,2022-11-24,16570.0
4,2022-11-25,16309.0
5,2022-11-26,16309.0
6,2022-11-27,16309.0


In [8]:
data = pd.concat((data,df),axis=0)
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date', ascending=True)
data = data[-2220:]
data

,date,도매가격
1706,2016-01-04,17343.0
1705,2016-01-05,18445.0
1704,2016-01-06,18813.0
1703,2016-01-07,18905.0
1702,2016-01-08,18950.0
...,...,...
2,2022-11-23,16951.0
3,2022-11-24,16570.0
4,2022-11-25,16309.0
5,2022-11-26,16309.0


In [9]:
# prophet은 ds,y라는 두가지 컬럼만 있어야 함(컬럼 이름 맞추기)
data = data[['date', '도매가격']]
data = data.rename(columns=({'date':'ds','도매가격':'y'}))

In [10]:
Gujeyeok = pd.DataFrame({
'holiday': 'Gujeyeok',
'ds': pd.to_datetime(['2017-02-07', '2017-02-09', '2017-02-13',
                      '2017-02-14', '2018-03-27', '2018-04-03',
                      '2019-01-28', '2019-01-29', '2019-01-31'
                      ]),
'lower_window': 0,
'upper_window': 1,
})

In [11]:
m = Prophet(holidays=Gujeyeok, changepoint_range=0.9, changepoint_prior_scale=0.5, yearly_seasonality=20)
m.add_seasonality(name='monthly', period=30, fourier_order=5)
m.add_country_holidays(country_name='KR').fit(data)
future = m.make_future_dataframe(periods=4,freq='W') # 주 단위로, 4주 가격 예측 실시
forecast = m.predict(future)

15:31:56 - cmdstanpy - INFO - Chain [1] start processing
15:31:57 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
forecast =  forecast[['ds','yhat']].tail()
forecast

,ds,yhat
1731,2022-11-27,17290.354087
1732,2022-12-04,17862.151291
1733,2022-12-11,17289.661885
1734,2022-12-18,16298.193606
1735,2022-12-25,16620.680952
